In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import os

os.chdir('..')

In [ ]:
from google.colab import files
uploaded = files.upload()
#questions = pd.read_csv('./MLResearchProject/Data_Sets/Questions_300.csv')

Saving untaggedA.csv to untaggedA.csv


In [ ]:
import io
questions_og = pd.read_csv(io.BytesIO(uploaded['untaggedA.csv']))

questions = questions_og

The following 3 blocks carry out the Pre-processing of the dataset

In [ ]:
questions['Question_Processed'] = questions['Question'].map(lambda x: x.lower())
data_og = questions.Question_Processed.values.tolist()

import re

#Applying this tokenisation approach to the entire input dataset
test_text = []
for sentence in data_og:
  token_punct = re.split('\s',sentence)
  test_text.append(token_punct)

#Defining function words to search for
words_funct = ['for', 'if', 'while', 'print', 'boolean', 'bigo', 'else', 'elseif' ]

#Removing function words when used outside of the appropriate context
text_processed = [[word for word in doc if word not in words_funct] for doc in test_text]

from nltk.tokenize.treebank import TreebankWordDetokenizer

#Reversing the process of tokenisation
questions_testproc = []
for sentence in text_processed:
  detoken = TreebankWordDetokenizer().detokenize(sentence)
  questions_testproc.append(detoken)

from pandas import DataFrame
questions_proc = DataFrame(questions_testproc,columns=['Questions_Processed'])

questions_proc['Question_Processed'] = questions_proc['Questions_Processed'].map(lambda x: re.sub('[,\.!?]', '', x))
questions_proc.index = questions.index
questions['Question_Processed'] = questions_proc['Question_Processed']

#Tokenising sentences into individual words with no punctuation
#Note: This process tends to remove any mathematical arguments from the questions 
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        #deacc=True removes accent marks

data = questions.Question_Processed.values.tolist()
data_words = list(sent_to_words(data))

#Forming Bigrams and Trigrams
bigram = gensim.models.Phrases(data_words, min_count = 5, threshold = 100)
trigram = gensim.models.Phrases(bigram[data_words], threshold = 100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Defining functions for processing

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

words_keep = ['for', 'if', 'while', 'print', 'Boolean', 'BigO', 'else', 'elseif' ]
new_stop_words = set(stop_words).difference(words_keep)
newer_stop_words = [word for word in stop_words if word not in words_keep] 

#Define functions for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in new_stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV','SCONJ','ADP','PROPN','X']):
  """https://spacy.io/api/annotation"""
  texts_out = []
  texts_post = []
  for sent in texts:
      doc = nlp(" ".join(sent))
      texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      #texts_post.append([token.pos for token in doc])
      #print(texts_out)
  return texts_out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Stopword Removal and Lemmatization

In [ ]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Form Trigrams
#data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable = ['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV','SCONJ','ADP','PROPN','X'])

import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
from gensim.models import HdpModel

Defining functions to produce results

In [ ]:
def hdp_vals(n,corpus_set,id2word_set):
  hdp_model = HdpModel(corpus=corpus_set,id2word=id2word_set,T=n)
  alpha_vec = hdp_model.hdp_to_lda()[0]

  x = len(alpha_vec[alpha_vec>1/n])
  y = len(alpha_vec[alpha_vec>1/x])
  z = len(alpha_vec[alpha_vec>1/y])

  return(x,y,z)

def formatted_table(lda_model,corpus,data_lemmatized):
  df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_lemmatized)

  # Format
  df_dominant_topic = df_topic_sents_keywords.reset_index()
  df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
  df_dominant_topic['Question'] = questions['Question']

  df_dominant_topic.sort_values("Dominant_Topic", axis = 0, ascending = True, 
                  inplace = True, na_position ='last')
  return(df_dominant_topic)

def get_used_topics(n,corpus,data_lemmatized,id2word):
  lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=n, random_state=100, chunksize=100, passes=10, alpha=0.31, eta=0.41)
  table = formatted_table(lda_model,corpus,data_lemmatized)
  
  index_list = list(table.index)
  topics_used = []
    
  for i in index_list:
    topics_used.append(table.at[i,'Dominant_Topic'])

  x = len(set(topics_used))

  return(x)

def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
import tqdm
import time
import shutil

In [ ]:
def get_averages(vector,runs=100):

  #Establishing vectors to hold average results
  topic_final_vec = []
  average_topics = []

  #Establish vectors to hold the Run details for each Question set as DataFrame objects
  run_dat = []
  length_dat = []

  #Progess Bar
  pbar = tqdm.tqdm(total=len(vector))

  for i in vector:
    #Start timer
    start = time.process_time()

    #Establish vector to hold the details of each run as Dataframe objects
    outputs = []

    #Establishing the appropriate sets to use for each step
    data_lemmatized_set = data_lemmatized[:i]
    corpus_set = corpus[:i]
    id2word_set = corpora.Dictionary(data_lemmatized_set)

    #Establishing vectors to hold the runs results
    topic_vec = []
    ratio_vec = []
    length_vec = []

    for k in list(range(0,runs,1)):
      #Compute the value of HDP(2)
      x = hdp_vals(i,corpus_set,id2word_set)[1]

      #Establish vector to hold topic numbers for the run
      y = []
      y.append(x)

      #Establish vector to hold ratios for the run
      z = []

      #Initialise ratio, topic number and length to start the run
      ratio = 0
      x1 = x
      n = 0

      #Iteration Process
      while ratio<1 and n<3 :
        #Determine the number of topics used and the new ratio
        t = get_used_topics(x1,corpus_set,data_lemmatized_set,id2word_set)
        ratio = t/x1
        
        #Update the topic number and length; Record the ratio
        x1 = t
        z.append(ratio)
        n += 1

        #Check to end the loop
        if ratio == 1 or n == 3 :
          break

        #Record the new number of topics (if there is one)
        y.append(t)

      #Recording values to the appropriate vector for average details
      topic_final_vec.append(y[-1])

      #Recording values to the appropriate vector for run details
      topic_vec.append(y)
      ratio_vec.append(z)
      length_vec.append(n)

      #Recording run details as a DataFrame object
      outputs.append(DataFrame(data={'Topic':topic_vec[k],'Ratio':ratio_vec[k]}))

    #Record Run Lengths and make a DataFrame object
    output_len = {'Run':list(range(1,(runs+1),1)),'Length':length_vec}
    results2 = DataFrame(data=output_len)

    #Record Length results to the appropriate vector
    length_dat.append(results2)

    #Combine the Run results and Record to the appropriate vector
    results1 = pd.concat(outputs,axis=1)
    run_dat.append(results1)

    #Determine and record the average topic number for the runs
    val_to_round = sum(topic_final_vec)/len(topic_final_vec) 
    average_topics.append(int(val_to_round))

    #Print timer and update progress bar
    print(time.process_time() - start)
    pbar.update(1)    

  pbar.close()
  return(average_topics,run_dat,length_dat)

In [ ]:
#Set vector of question numbers to test
n_tot = len(corpus)
n_end = n_tot // 100

q_uest_list = list(range(100,n_tot+1,100))
q_uest_list[n_end-1] = n_tot

#Partition the question list to account for runtime
q_ues_1 = q_uest_list[:5]
q_ues_2 = q_uest_list[5:8]
q_ues_3 = q_uest_list[8:11]
q_ues_4 = q_uest_list[11:13] 

#Check
#print(q_uest_list)
print(q_ues_1,q_ues_2,q_ues_3,q_ues_4)
#print(q_ues_1+q_ues_2+q_ues_3+q_ues_4)

[100, 200, 300, 400, 500] [600, 700, 800] [900, 1000, 1100] [1200, 1303]


In [ ]:
[avgs_1,runs_1,lengths_1] = get_averages(q_ues_1)

#Upload the Path and Length data to Drive (specify location)
for i in list(range(0,len(q_ues_1),1)):
  runs = runs_1[i]
  lengths = lengths_1[i]

  filename1 = 'UntaggedA_Paths_' + str(q_ues_1[i]) + '.csv'
  filename2 = 'UntaggedA_Lengths_' + str(q_ues_1[i]) + '.csv'

  runs.to_csv(filename1)
  lengths.to_csv(filename2)

  shutil.copy2(filename1, '/content/drive/My Drive/MLResearchProject/Average Termination Untagged')
  shutil.copy2(filename2, '/content/drive/My Drive/MLResearchProject/Average Termination Untagged')  

#Upload Average(s) data to Drive (specify location) 
output_1 = {'Questions Used':q_ues_1,'Average Topic Number Found':avgs_1}
avgs_output_1 = DataFrame(data=output_1)
avgs_output_1.to_csv('UntaggedA_Averages_1.csv')

!cp UntaggedA_Averages_1.csv "/content/drive/My Drive/MLResearchProject/Average Termination Untagged"



  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
[avgs_2,runs_2,lengths_2] = get_averages(q_ues_2)

#Upload the Path and Length data to Drive (specify location)
for i in list(range(0,len(ques_2),1)):
  runs = runs_2[i]
  lengths = lengths_2[i]

  filename1 = 'UntaggedA_Paths_' + str(q_ues_2[i]) + '.csv'
  filename2 = 'UntaggedA_Lengths_' + str(q_ues_2[i]) + '.csv'

  runs.to_csv(filename1)
  lengths.to_csv(filename2)

  shutil.copy2(filename1, '/content/drive/My Drive/MLResearchProject/Average Termination Untagged')
  shutil.copy2(filename2, '/content/drive/My Drive/MLResearchProject/Average Termination Untagged')  

#Upload Average(s) data to Drive (specify location) 
output_2 = {'Questions Used':q_ues_2,'Average Topic Number Found':avgs_2}
avgs_output_2 = DataFrame(data=output_2)
avgs_output_2.to_csv('UntaggedA_Averages_2.csv')

!cp UntaggedA_Averages_2.csv "/content/drive/My Drive/MLResearchProject/Average Termination Untagged"

In [ ]:
[avgs_3,runs_3,lengths_3] = get_averages(q_ues_3)

#Upload the Path and Length data to Drive (specify location)
for i in list(range(0,len(q_ues_3),1)):
  runs = runs_3[i]
  lengths = lengths_3[i]

  filename1 = 'UntaggedA_Paths_' + str(q_ues_3[i]) + '.csv'
  filename2 = 'UntaggedA_Lengths_' + str(q_ues_3[i]) + '.csv'

  runs.to_csv(filename1)
  lengths.to_csv(filename2)

  shutil.copy2(filename1, '/content/drive/My Drive/MLResearchProject/Average Termination Untagged')
  shutil.copy2(filename2, '/content/drive/My Drive/MLResearchProject/Average Termination Untagged')  

#Upload Average(s) data to Drive (specify location) 
output_3 = {'Questions Used':q_ues_3,'Average Topic Number Found':avgs_3}
avgs_output_3 = DataFrame(data=output_3)
avgs_output_3.to_csv('UntaggedA_Averages_3.csv')

!cp TaggedA_Averages_3.csv "/content/drive/My Drive/MLResearchProject/Average Termination Untagged"

 33%|███▎      | 1/3 [1:30:10<3:00:20, 5410.16s/it]

5865.186952907


 67%|██████▋   | 2/3 [3:29:24<1:38:53, 5933.49s/it]

7753.0007511329995


100%|██████████| 3/3 [5:36:17<00:00, 6725.85s/it]

8379.076947503


In [ ]:
[avgs_4,runs_4,lengths_4] = get_averages(q_ues_4)

#Upload the Path and Length data to Drive (specify location)
for i in list(range(0,len(q_ues_4),1)):
  runs = runs_4[i]
  lengths = lengths_4[i]

  filename1 = 'UntaggedA_Paths_' + str(q_ues_4[i]) + '.csv'
  filename2 = 'UntaggedA_Lengths_' + str(q_ues_4[i]) + '.csv'

  runs.to_csv(filename1)
  lengths.to_csv(filename2)

  shutil.copy2(filename1, '/content/drive/My Drive/MLResearchProject/Average Termination Untagged')
  shutil.copy2(filename2, '/content/drive/My Drive/MLResearchProject/Average Termination Untagged')  

#Upload Average(s) data to Drive (specify location) 
output_4 = {'Questions Used':q_ues_4,'Average Topic Number Found':avgs_4}
avgs_output_4 = DataFrame(data=output_4)
avgs_output_4.to_csv('TaggedA_Averages_4.csv')

!cp TaggedA_Averages_4.csv "/content/drive/My Drive/MLResearchProject/Average Termination Untagged"

 50%|█████     | 1/2 [2:24:13<2:24:13, 8653.53s/it]

9538.829588979


100%|██████████| 2/2 [4:31:00<00:00, 8130.10s/it]

8687.161869608997


In [ ]:
# output = {'Questions Used':q_uest_list,'Average Topic Number Found':avgs_1+avgs_2+avgs_3}
# results = DataFrame(data=output)
# results.to_csv('TaggedA.csv')

# !cp TaggedA.csv "/content/drive/My Drive/MLResearchProject/Average Termination"